# Lab 4

In the previous labs, we perform mortality prediction using DNN and CNN. However, to deal with sequential data, the most commonly used architecture is actually recurrent neural network (RNN). This lab introduces you to the motivation, implementation, and some commonly used RNN models. Let us get started!

Table of Contents:
- Motivation
- Implementation
- Modern RNN Models
- Assignment

Some contents of this lab are adapted from [Dive into Deep Learning](https://d2l.ai) and [Official PyTorch Tutorials](https://pytorch.org/tutorials/).

In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

os, random → standard Python tools.

numpy → math & arrays.

torch, torch.nn, torch.nn.functional → deep learning.

pandas → working with data tables.

In [2]:
# set seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

When training AI models, randomness happens everywhere — in data shuffling, weight initialization, augmentations, etc.
If you set the seed like this, your experiments are repeatable.
Example: If you train today and tomorrow with the same code and data, you get exactly the same results.

In [3]:
DATA_PATH = "../LAB4-lib/data"
assert os.path.isdir(DATA_PATH)
!ls {DATA_PATH}

data.csv


“Here’s the box path where I keep my datadoler. Check that it exists, if it , continue, then open it and tell me what’s inside.”

## 1. Motivation

While CNNs can efficiently process spatial information, recurrent neural networks (RNNs) are designed to better handle sequential information. RNNs introduce state variables to store past information, together with the current inputs, to determine the current outputs.

That is, let us say we have a sequence $x_1, x_2, \dots, x_t$. RNN attempts to model the conditional probability: $P(x_t \mid x_1, x_2, \dots, x_{t-1})$. More specifically, RNN leverages a hidden state variable  that stores the sequence information up to time step $t−1$:

$$P(x_t \mid x_{1}, x_{2}, \ldots, x_{t-1}) \approx P(x_t \mid h_{t-1}),$$

where $h_{t-1}$ is the hidden state variable. In general, the hidden state at any time step $t$ could be computed based on both the current input $x_t$ and the previous hidden state $h_{t-1}$:

$$h_t = f(x_{t}, h_{t-1}).$$ 

Here’s the short version of what I just explained:

CNNs are great for spatial data (like images) but don’t naturally handle sequences where order matters.

RNNs are designed for sequences — they remember past information using a hidden state.

They model the probability of the current input given all previous inputs-𝑃(𝑥𝑡∣𝑥1,𝑥2,…,𝑥𝑡−1)≈𝑃(𝑥𝑡∣ℎ𝑡−1),
where ℎ𝑡−1 is the hidden state variable. In general, the hidden state at any time step 𝑡 could be computed based on both the current input 𝑥𝑡 and the previous hidden state ℎ𝑡−1

:

ℎ𝑡=𝑓(𝑥𝑡,ℎ𝑡−1).


## 2. Implementation

Assume that we have a minibatch of inputs $\mathbf{X}_t \in \mathbb{R}^{n \times d}$ at time step $t$. In other words, for a minibatch of $n$ sequence examples, each row of $\mathbf{X}_t$ corresponds to one example at time step $t$ from the sequence. Next, denote by $\mathbf{H}_t \in \mathbb{R}^{n \times h}$ the hidden variable of time step $t$. Further, we save the hidden variable $\mathbf{H}_{t-1}$ from the previous time step and introduce a new weight parameter $\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$ to describe how to use the hidden variable of the previous time step in the current time step. Specifically, the calculation of the hidden variable of the current time step is determined by the input of the current time step together with the hidden variable of the previous time step:

$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h).$$

From the relationship between hidden variables $\mathbf{H}_{t}$ and $\mathbf{H}_{t-1}$ of adjacent time steps, we know that these variables captured and retained the sequence’s historical information up to their current time step, just like the state or memory of the neural network’s current time step. Therefore, such a hidden variable is called a hidden state. Since the hidden state uses the same definition of the previous time step in the current time step, the computation of $\mathbf{H}_{t}$ is recurrent. Hence, neural networks with hidden states based on recurrent computation are named recurrent neural networks. Layers that perform the computation of $\mathbf{H}_{t}$ in RNNs are called recurrent layers.

For time step $t$, the output of the output layer is similar to the computation in the MLP:

$$\mathbf{O}_t = \mathbf{H}_t \mathbf{W}_{hq} + \mathbf{b}_q.$$

Parameters of the RNN include the weights $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}, \mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$, and the bias $\mathbf{b}_h \in \mathbb{R}^{1 \times h}$ of the hidden layer, together with the weights $\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$ and the bias $\mathbf{b}_q \in \mathbb{R}^{1 \times q}$ of the output layer. It is worth mentioning that even at different time steps, RNNs always use these model parameters. Therefore, the parameterization cost of an RNN does not grow as the number of time steps increases.

<img src='img/rnn.svg'>

Inputs and Hidden States

At each time step 
t
t, the RNN takes:

The current input 
Xt
X
t
	​

 (data at this time step)

The hidden state 
Ht−1
H
t−1
	​

 from the previous time step (memory from the past)

Updating the Hidden State

The new hidden state 
Ht
H
t
	​

 is calculated by combining:

The current input 
Xt
X
t
	​

 multiplied by input weights 
Wxh
W
xh
	​


The previous hidden state 
Ht−1
H
t−1
	​

 multiplied by recurrent weights 
Whh
W
hh
	​


A bias term 
bh
b
h
	​


Then applying an activation function 
ϕ
ϕ (e.g., tanh or ReLU).

Formula:

Ht=ϕ(XtWxh+Ht−1Whh+bh)
H
t
	​

=ϕ(X
t
	​

W
xh
	​

+H
t−1
	​

W
hh
	​

+b
h
	​

)

Why It’s Called “Recurrent”

The same formula is used at every time step.

The hidden state carries historical information through the sequence — acting like memory.

Output at Each Step

The RNN produces an output 
Ot
O
t
	​

 from the hidden state:

Ot=HtWhq+bq
O
t
	​

=H
t
	​

W
hq
	​

+b
q
	​


Parameters Are Shared Across Time

The weights and biases (
Wxh,Whh,Whq,bh,bq
W
xh
	​

,W
hh
	​

,W
hq
	​

,b
h
	​

,b
q
	​

) stay the same for every time step.

This makes RNNs efficient because the number of parameters doesn’t grow with sequence length.

### Exercise 1 [20 points]

Implement the function calculating the current hidden state:

$$\mathbf{H}_t = \mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h.$$

Note that here `X` is the input at a sinlge time step.

In [4]:
def calculate_current_h(X, H, W_xh, W_hh, b_h):
    """
    Params:
        - X: (batch size, input dimension)
        - H: (batch size, hidden dimension)
        - W_xh: (input dimension, hidden dimension)
        - W_hh: (hidden dimension, hidden dimension)
        - b_h: (1, hidden dimension)
    """
    # your code here
    H_t = X @ W_xh + H @ W_hh + b_h
    #raise NotImplementedError
    return H_t

In [5]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

torch.manual_seed(42)

n, d, h = 2, 8, 4 # batch size, input dimension, hidden dimension
X, W_xh = torch.normal(0, 1, (n, d)), torch.normal(0, 1, (d, h))
H, W_hh = torch.normal(0, 1, (n, h)), torch.normal(0, 1, (h, h))
b_h = torch.normal(0, 1, (1, h))
assert torch.allclose(calculate_current_h(X, H, W_xh, W_hh, b_h),
                      torch.tensor([[ 5.1226, -0.6884,  3.1821,  6.6513],
                                    [-0.6077,  2.2313, -1.4812,  0.4403]]), rtol=1e-2)



### Exercise 2 [20 points]

Call the previous implemented `calculate_current_h` recursively to calculate the final hidden state.

Note that here `inputs` is the entire sequence.

In [6]:
def rnn(inputs, state, W_xh, W_hh, b_h):
    """
    Params:
        - inputs: (batch size, sequence length, input dimension)
        - state: (batch size, hidden dimension)  # initial hidden state H0
        - W_xh: (input dimension, hidden dimension)
        - W_hh: (hidden dimension, hidden dimension)
        - b_h: (1, hidden dimension)
    Returns:
        - final hidden state after processing the sequence
    """
    H = state  # start from initial hidden state

    # Loop through each time step
    for t in range(inputs.shape[1]):
        X_t = inputs[:, t, :]  # (batch size, input dimension)
        H = X_t @ W_xh + H @ W_hh + b_h  # update hidden state

    return H  # final hidden state


In [7]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

torch.manual_seed(42)

n, t, d, h = 2, 5, 8, 4 # batch size, sequence length, input dimension, hidden dimension

inputs = torch.normal(0, 1, (n, t, d))
initial_state  = torch.normal(0, 1, (n, h))

W_xh = torch.normal(0, 1, (d, h))
W_hh = torch.normal(0, 1, (h, h))
b_h = torch.normal(0, 1, (1, h))
assert torch.allclose(rnn(inputs, initial_state, W_xh, W_hh, b_h),
                      torch.tensor([[ -46.5894,  -61.1859,   14.3644,   56.4997],
                                    [ 166.5581,   84.0468,  -24.5756, -149.2971]]), rtol=1e-2)



## 3. Modern RNN Models

We have introduced the basics of RNNs, which can better handle sequence data. However, such techniques may not be sufficient for practitioners when they face a wide range of sequence learning problems nowadays.

For instance, a notable issue in practice is the numerical instability of RNNs. Although there are several implementation tricks such as gradient clipping, this issue can be alleviated further with more sophisticated designs of sequence models. Specifically, gated RNNs are much more common in practice. In this section, we will introduce you two of such widely-used networks, namely long short-term memory (LSTM) and gated recurrent units (GRUs).

Here, we will only cover the basic concepts of GRU and LSTM. We won't discuss the architecture in details. If you are intereseted, you can refer to this [link](https://d2l.ai/chapter_recurrent-modern/index.html).

### 3.1 Gated Recurrent Units (GRU)

The key distinction between vanilla RNNs and GRUs is that the latter support gating of the hidden state. This means that we have dedicated mechanisms for when a hidden state should be updated and also when it should be reset. These mechanisms are learned and the network can selectively keep or forget information. For instance, if the first token is of great importance we will learn not to update the hidden state after the first observation. Likewise, we will learn to skip irrelevant temporary observations.

<img src='img/gru.svg'>

As shown in the figure above, the reset gates help capture short-term dependencies in sequences. And the update gates help capture long-term dependencies in sequences.

To initialize a [GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html#torch.nn.GRU) layer in PyTorch, try the following snippet:

In [8]:
n, t, d, h = 2, 5, 8, 4 # batch size, sequence length, input dimension, hidden dimension

# If batch_first=True, then the input and output tensors are 
# provided as (batch, seq, feature) instead of (seq, batch, feature).
rnn = nn.GRU(input_size=d, hidden_size=h, batch_first=True)

inputs = torch.randn(n, t, d)
h0 = torch.randn(1, n, h) # the first dimension is the number of RNN layers (default value is 1)

output, hn = rnn(inputs, h0)
print('output:', output.shape)
print('hn:', hn.shape)

output: torch.Size([2, 5, 4])
hn: torch.Size([1, 2, 4])


GRU Summary in PyTorch

Inputs:

inputs: (batch_size, seq_len, input_dim) → (2, 5, 8)

h0: (num_layers, batch_size, hidden_dim) → (1, 2, 4)

Outputs:

output: hidden states for all time steps → (2, 5, 4)

hn: hidden state from the last time step → (1, 2, 4)

Intuition:

output = sequence info (good for tasks needing predictions at every step).

hn = summary of the sequence (good for tasks like classification).

### 3.2 Long Short-Term Memory (LSTM)

The challenge to address long-term information preservation and short-term input skipping in latent variable models has existed for a long time. One of the earliest approaches to address this was the long short-term memory (LSTM) ([Hochreiter & Schmidhuber, 1997](https://direct.mit.edu/neco/article/9/8/1735/6109/Long-Short-Term-Memory)). It shares many of the properties of the GRU. Interestingly, LSTMs have a slightly more complex design than GRUs but predates GRUs by almost two decades.

LSTM introduces a memory cell (or cell for short) that has the same shape as the hidden state, engineered to record additional information. To control the memory cell we need a number of gates. One gate is needed to read out the entries from the cell. We will refer to this as the output gate. A second gate is needed to decide when to read data into the cell. We refer to this as the input gate. Last, we need a mechanism to reset the content of the cell, governed by a forget gate. The motivation for such a design is the same as that of GRUs, namely to be able to decide when to remember and when to ignore inputs in the hidden state via a dedicated mechanism.

<img src='img/lstm.svg'>

To initialize a [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) layer in PyTorch, try the following snippet:

In [9]:
n, t, d, h = 2, 5, 8, 4 # batch size, sequence length, input dimension, hidden dimension

# If batch_first=True, then the input and output tensors are 
# provided as (batch, seq, feature) instead of (seq, batch, feature).
rnn = nn.LSTM(input_size=d, hidden_size=h, batch_first=True)

inputs = torch.randn(n, t, d)
h0 = torch.randn(1, n, h) # the first dimension is the number of RNN layers (default value is 1)
c0 = torch.randn(1, n, h)

output, (hn, cn) = rnn(inputs, (h0, c0))
print('output:', output.shape)
print('hn:', hn.shape)
print('cn:', cn.shape)

output: torch.Size([2, 5, 4])
hn: torch.Size([1, 2, 4])
cn: torch.Size([1, 2, 4])


Modern RNN Models
Why do we need them?

Vanilla RNNs suffer from:

Numerical instability (gradients explode/vanish).

Difficulty learning long-term dependencies.

Solutions:

Implementation tricks: gradient clipping, careful initialization.

Architectural improvements: Gated RNNs (GRU & LSTM).

🔹 3.1 Gated Recurrent Units (GRU)
Core Idea

GRUs add gates to the RNN hidden state.

Gates = learnable mechanisms that decide:

When to update the hidden state.

When to reset the hidden state.

This allows GRUs to selectively keep or forget information.

Intuition

Update gate → Should we carry forward old information or replace it with new input?

Reset gate → Should we ignore the past and reset memory for the next state?

Example

If the first token is very important (like the subject of a sentence), GRU can learn to not update the hidden state too much afterward.

If the model encounters irrelevant tokens, it can reset and ignore them.

✅ Key takeaway:
GRUs are more efficient than vanilla RNNs because they can adaptively remember or forget, making them good at handling longer sequences without blowing up or losing gradients.

Modern RNN Models
Why do we need them?

Vanilla RNNs suffer from:

Numerical instability (gradients explode/vanish).

Difficulty learning long-term dependencies.

Solutions:

Implementation tricks: gradient clipping, careful initialization.

Architectural improvements: Gated RNNs (GRU & LSTM).

🔹 3.1 Gated Recurrent Units (GRU)
Core Idea

GRUs add gates to the RNN hidden state.

Gates = learnable mechanisms that decide:

When to update the hidden state.

When to reset the hidden state.

This allows GRUs to selectively keep or forget information.

Intuition

Update gate → Should we carry forward old information or replace it with new input?

Reset gate → Should we ignore the past and reset memory for the next state?

Example

If the first token is very important (like the subject of a sentence), GRU can learn to not update the hidden state too much afterward.

If the model encounters irrelevant tokens, it can reset and ignore them.

✅ Key takeaway:
GRUs are more efficient than vanilla RNNs because they can adaptively remember or forget, making them good at handling longer sequences without blowing up or losing gradients.

| Feature     | GRU                    | LSTM                              |
| ----------- | ---------------------- | --------------------------------- |
| Gates       | 2 (update, reset)      | 3 (forget, input, output)         |
| Memory Cell | No (hidden state only) | Yes (hidden + cell state)         |
| Complexity  | Simpler, faster        | More complex, slower              |
| Performance | Good for many tasks    | Better for long-term dependencies |


## Assignment [60 points]

In this assignment, you will use [MIMIC-III Demo](https://physionet.org/content/mimiciii-demo/) dataset, which contains all intensive care unit (ICU) stays for 100 patients. The task is Mortality Prediction.

### Load Data

In the previous lab, we have preprocessed the data. Thus, for this lab, we will directly use the processed data.

In [10]:
!ls {DATA_PATH}

data.csv


Here are the helper fuctions and CustomDataset from the previous lab. 

We will use the entire patient visit instead of only the last visit.

Note that in this lab, we **do not** need to exclude patients with only one visit.

In [11]:
# two helper functions

TOTAL_NUM_CODES = 271


def read_csv(filename):
    """ reading csv from filename """
    data = []
    with open(filename, "r") as file:
        csv_reader = csv.DictReader(file, delimiter=',')
        for row in csv_reader:
            data.append(row)
    header = list(data[0].keys())
    return header, data


def to_one_hot(label, num_class):
    """ convert to one hot label """
    one_hot_label = [0] * num_class
    for i in label:
        one_hot_label[i] = 1
    return one_hot_label

read_csv(filename) → Reads a CSV file and returns:

header: list of column names.

data: list of row dictionaries.

to_one_hot(label, num_class) → Converts label indices into a one-hot vector of length num_class.

👉 read_csv = data loader, to_one_hot = label encoder.

In [12]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    
    def __init__(self):
        # read the csv
        self._df = pd.read_csv(f'{DATA_PATH}/data.csv')
        # split diagnosis code index by ';' and convert it to integer
        self._df.icd9 = self._df.icd9.apply(lambda x: [int(i) for i in x.split(';')])
        # build data dict
        self._build_data_dict()
        # a list of subject ids
        self._subj_ids = list(self._data.keys())
        # sort the subject ids to maintain a fixed order
        self._subj_ids.sort()
    
    def _build_data_dict(self):
        """ 
        build SUBJECT_ID to ADMISSION dict
            - subject_id
                - icd9: a list of ICD9 code index
                - mortality: 0/1 morality label
        """
        dict_data = {}
        df = self._df.groupby('subject_id').agg({'mortality': lambda x: x.iloc[0], 'icd9': list}).reset_index()
        for idx, row in df.iterrows():
            subj_id = row.subject_id
            dict_data[subj_id] = {}
            dict_data[subj_id]['icd9'] = row.icd9
            dict_data[subj_id]['mortality'] = row.mortality
        self._data = dict_data
    
    def __len__(self):
        """ return the number of samples (i.e. patients). """
        return len(self._subj_ids)
    
    def __getitem__(self, index):
        """ generates one sample of data. """
        # obtain the subject id
        subj_id = self._subj_ids[index]
        # obtain the data dict by subject id
        data = self._data[subj_id]
        # convert last admission's diagnosis code index to one hot
        x = torch.tensor([to_one_hot(visit, TOTAL_NUM_CODES) for visit in data['icd9']], dtype=torch.float32)
        # mortality label
        y = torch.tensor(data['mortality'], dtype=torch.float32)
        return x, y

subject_id,mortality,icd9
1,0,12;45;78
1,0,7;19
2,1,4;5
3,0,8
Patient 1 has 2 admissions: [12,45,78] and [7,19]

Patient 2 has 1 admission: [4,5]

Patient 3 has 1 admission: [8]


🔄 How CustomDataset processes this

Groups by subject_id.

Keeps the first mortality label for each patient.

Collects all admissions for that patient.

So internally it builds:
_data = {
  1: { "icd9": [[12,45,78], [7,19]], "mortality": 0 },
  2: { "icd9": [[4,5]], "mortality": 1 },
  3: { "icd9": [[8]], "mortality": 0 }
}
🔑 What happens in __getitem__(index)?

Suppose TOTAL_NUM_CODES = 10 for simplicity.

If we call dataset[0] → Patient 1
x = [
  to_one_hot([12,45,78], 10),   # (these indices truncated in toy case)
  to_one_hot([7,19], 10)
]
y = 0



Output for dataset[0] (Patient 1)

x (admissions, one-hot encoded):
Output for dataset[0] (Patient 1)

[[0,1,0,0,1,0,0,1,0,0],   # admission 1 → codes 1,4,7
 [0,0,1,0,0,0,1,0,0,0]]   # admission 2 → code 7, 19


In [13]:
dataset = CustomDataset()
print('Size of dataset:', len(dataset))

Size of dataset: 99


In [14]:
from torch.utils.data.dataset import random_split


split = int(len(dataset)*0.7)

lengths = [split, len(dataset) - split]
train_dataset, test_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))

Length of train dataset: 69
Length of test dataset: 30


Here is an example of $x$, and $y$. 

In [15]:
x, y = train_dataset[0]
print(f'Example x (shape {x.shape}):\n', x)
print(f'Example y:\n', y)

Example x (shape torch.Size([1, 271])):
 tensor([[0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

We can see that $x$ is of shape $(2, 271)$, which means there are $271$ diagnosis codes in total, and this patient has two visits. It is in one-hot format. A $1$ in position $i$ means that diagnosis code of index $i$ appears in the that visit.

And $y$ is either $0$ or $1$.

### Padding [20 points]

In the previous lab, we implement a collate function `collate_fn()` to pad the sequence into the same length. For RNN, we will do something similar.

Moreover, we will keep a separate variable storing the length of each sequence. Later, we will use this length variable to select the mask out padding visits.

In [16]:
def collate_fn(data):
    """
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
        sequences to the sample shape (max # visits, total # diagnosis codes). Further, you need to store 
        the true length of each sequence into l.
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patients, total # diagnosis codes, max # visits) of type torch.float
        y: a tensor of shape (# patients) of type torch.float
        l: a tensor of shape (# patients) of type torch.long
        
    Note that you can obtains the list of diagnosis codes and the list of mortality labels
        using: `sequences, labels = zip(*data)`
    """

    sequences, labels = zip(*data)

    y = torch.tensor(labels, dtype=torch.float)
    
    num_patients = len(sequences)
    num_visits = [patient.shape[0] for patient in sequences]
    total_num_codes = sequences[0].shape[1]

    max_num_visits = max(num_visits)
    
    x = torch.zeros((num_patients, max_num_visits, total_num_codes), dtype=torch.float)
    l = None
    
    # your code here
    # Pad each patient's sequence to max_num_visits
    x = torch.zeros((num_patients, max_num_visits, total_num_codes), dtype=torch.float)
    for i, seq in enumerate(sequences):
        n_vis = seq.shape[0]
        x[i, :n_vis, :] = torch.tensor(seq, dtype=torch.float)

    # Lengths as tensor (number of visits per patient)
    l = torch.tensor(num_visits, dtype=torch.long)
    
    return x, y, l
   

You need a collate_fn when your data samples are not simple, fixed-size tensors, or when you want custom logic for batching. For your current data (single fixed-size vector and label), the default is probably fine—but if you change your data structure, you'll need to write your own collate_fn to tell the DataLoader how to assemble a batch from individual items.



Example:
# Sample data: (sequence, label) pairs
# Patient 1: 2 visits, 5 codes per visit
seq1 = np.array([
    [1, 0, 0, 0, 1],  # visit 1
    [0, 1, 1, 0, 0],  # visit 2
])
label1 = 1

# Patient 2: 3 visits
seq2 = np.array([
    [0, 1, 0, 1, 0],  # visit 1
    [1, 0, 0, 0, 1],  # visit 2
    [0, 0, 1, 1, 0],  # visit 3
])
label2 = 0

# Patient 3: 1 visit
seq3 = np.array([
    [0, 0, 0, 1, 1],  # visit 1
])
label3 = 1

data = [
    (seq1, label1),
    (seq2, label2),
    (seq3, label3),
]



Output of coallet func
Output explained

    x: shape (3, 3, 5) — 3 patients, padded to 3 visits, 5 codes per visit.
    y: tensor of shape (3,) — [1., 0., 1.]
    l: tensor of shape (3,) — [2, 3, 1] (number of visits per patient)

Printed output:
Code

x shape: torch.Size([3, 3, 5])
tensor([
  [[1., 0., 0., 0., 1.],
   [0., 1., 1., 0., 0.],
   [0., 0., 0., 0., 0.]],    # Patient 1 (padded)
  [[0., 1., 0., 1., 0.],
   [1., 0., 0., 0., 1.],
   [0., 0., 1., 1., 0.]],    # Patient 2 (no pad)
  [[0., 0., 0., 1., 1.],
   [0., 0., 0., 0., 0.],
   [0., 0., 0., 0., 0.]]     # Patient 3 (padded)
])
y: tensor([1., 0., 1.])
l: tensor([2, 3, 1])

Loop explanation:

Example

Suppose:

    max_num_visits = 5
    total_num_codes = 3

Patient 1 has 2 visits:
Python

seq = [[1,0,0],[0,1,0]]
n_vis = 2
x[0, :2, :] = [[1,0,0],[0,1,0]]
# x[0, 2:, :] = [[0,0,0],[0,0,0],[0,0,0]]  # padding

Patient 2 has 5 visits:
Python

seq = [[1,0,0],[0,1,0],[0,0,1],[1,1,0],[0,0,1]]
n_vis = 5
x[1, :5, :] = seq
# no padding necessary

Summary

    This loop populates the batch tensor x with each patient’s visit data, left-aligning it and padding with zeros so all patients have the same number of visits (max in the batch).
    This is essential for batching variable-length sequences for models like RNNs or transformers.



In [17]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

from torch.utils.data import DataLoader


loader = DataLoader(train_dataset, batch_size=4, collate_fn=collate_fn)
loader_iter = iter(loader)
x, y, l = next(loader_iter)

print(x.shape)
print(y.shape)
print(l)

assert x.dtype == torch.float
assert y.dtype == torch.float
assert l.dtype == torch.long

assert x.shape[0] == 4
assert x.shape[-1] == 271
assert y.shape == (4,)
assert l.shape == (4,)

for i in range(4):
    real_x, real_y = train_dataset[i]
    assert len(real_x) == l[i]
    for j in range(real_x.shape[0]):
        visit = real_x[j]
        got = x[i, j, :]
        assert all(visit == got)
        assert real_y == y[i]



torch.Size([4, 3, 271])
torch.Size([4])
tensor([1, 3, 1, 1])


### Data Loader

Now, we can load the dataset into the data loader.

In [18]:
from torch.utils.data import DataLoader

# how many samples per batch to load
batch_size = 4

# prepare dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

print("# of train batches:", len(train_loader))
print("# of test batches:", len(test_loader))

# of train batches: 18
# of test batches: 8


In [19]:
train_iter = iter(train_loader)
x, y, l = next(train_iter)

print('Shape of a batch x:', x.shape)
print('Shape of a batch y:', y.shape)
print('Shape of a batch l:', l.shape)

Shape of a batch x: torch.Size([4, 1, 271])
Shape of a batch y: torch.Size([4])
Shape of a batch l: torch.Size([4])


This test ensures your batch loader:

    Pads visit sequences correctly.
    Tracks the correct number of visits.
    Produces the right types/shapes for modeling.
    Preserves the original data and labels for each batch element.


### Build the Model

<img src='img/naive-rnn.png'>

We will construct this simple RNN structure. So each input is a one-hot vector. At the 0-th visit, this has $\boldsymbol{X}_0$, and at t-th visit, this has $\boldsymbol{X}_t$.

Each one of them will then map to a hidden state $\boldsymbol{h}_t$. The hidden state $\boldsymbol{h}_t$ can be determined by $\boldsymbol{h}_{t-1}$ and the corresponding current input $\boldsymbol{X}_t$.

Finally, once we have the $\boldsymbol{h}_T$, the hidden state of the last timestamp, then we can use this as feature vectors and train a NN to perform the classification.

Now, let us build this model. The forward steps will be:

    1. Pass the inputs through the RNN layer;
    2. Obtain the hidden state at the last visit;
    3. Pass the hidden state through the linear and activation layers.

#### Mask Selection [20 points]

Importantly, you need to use `length` to mask out the paddings in step 2.

In [20]:
def get_last_visit(hidden_states, length):
    """
    TODO: obtain the hidden state for the last true visit (not padding visits)

    Arguments:
        hidden_states: the hidden states of each visit of shape (batch_size, # visits, hidden_dim)
        length: the true visit length of shape (batch_size,)

    Outputs:
        last_hidden_state: the hidden state for the last true visit of shape (batch_size, hidden_dim)
        
    NOTE: DO NOT use for loop.
    """
    
    # your code here
    idx = (length - 1).unsqueeze(1).unsqueeze(2)    # shape: (batch, 1, 1)
    idx = idx.expand(-1, 1, hidden_states.size(2))  # shape: (batch, 1, hidden_dim)
    last_hidden_state = hidden_states.gather(1, idx).squeeze(1)  # shape: (batch, hidden_dim)
    return last_hidden_state
    

import torch

hidden_states = torch.tensor([
    [[10, 11], [12, 13], [14, 15], [16, 17]],
    [[20, 21], [22, 23], [24, 25], [26, 27]],
    [[30, 31], [32, 33], [34, 35], [36, 37]],
])
length = torch.tensor([2, 4, 3])

idx = (length - 1).unsqueeze(1).unsqueeze(2).expand(-1, 1, hidden_states.size(2))
print("Index tensor for gather:\n", idx)

selected = hidden_states.gather(1, idx)
print("Gathered hidden states (with singleton dimension):\n", selected)

last_hidden_state = selected.squeeze(1)
print("Final last_hidden_state tensor:\n", last_hidden_state)



Each step  calculation
#idx
Let’s use your specific example:  
You have a tensor: `length - 1 = tensor([1, 3, 2])` (for 3 patients).

Here’s how `.unsqueeze(1)`, `.unsqueeze(2)`, and `.expand(-1, 1, hidden_dim)` work:

---

## Step-by-step with shapes and values

### 1. Start with
```python
idx = torch.tensor([1, 3, 2])           # Shape: (3,)
```
#### Values:
```
[1, 3, 2]
```

---

### 2. `.unsqueeze(1)`  
Adds a dimension at position 1:
```python
idx = idx.unsqueeze(1)                  # Shape: (3, 1)
```
#### Values:
```
[[1],
 [3],
 [2]]
```

---

### 3. `.unsqueeze(2)`  
Adds another dimension at position 2:
```python
idx = idx.unsqueeze(2)                  # Shape: (3, 1, 1)
```
#### Values:
```
[[[1]],
 [[3]],
 [[2]]]
```

---

### 4. `.expand(-1, 1, hidden_dim)`  
Suppose `hidden_dim = 2` (it could be any number, but let's use 2 for clarity).

- `.expand(-1, 1, 2)` means:
  - Keep the same size in the first dimension (`3` patients)
  - Keep the same size in the second dimension (`1`)
  - Expand the third dimension to `2` (hidden_dim)

```python
idx = idx.expand(-1, 1, 2)              # Shape: (3, 1, 2)
```
#### Values:
```
[[[1, 1]],
 [[3, 3]],
 [[2, 2]]]
```

---

## **What does this mean?**

- Each patient now has a row of the form `[[index, index]]`, where `index` is the visit you want for that patient, and it’s repeated for each hidden dimension.
- This shape and value matches what PyTorch’s `.gather()` expects:  
  You want to select, for each patient, the row (visit) at their specific index, and all hidden_dim values for that row.

---

## **In summary:**
- `.unsqueeze(1)` changes shape from `(3,)` to `(3,1)`
- `.unsqueeze(2)` changes shape from `(3,1)` to `(3,1,1)`
- `.expand(-1,1,2)` changes shape from `(3,1,1)` to `(3,1,2)` and repeats each index for the hidden dimension.

**This prepares your index tensor so you can batch-select each patient’s last (non-padding) hidden state across all hidden units at once.**

---

### **Visual Table**

| Patient | index | after expand (hidden_dim=2) |
|---------|-------|-----------------------------|
|    0    |   1   | [[1, 1]]                   |
|    1    |   3   | [[3, 3]]                   |
|    2    |   2   | [[2, 2]]                   |

---
#selected
Step 1: Gather
What does hidden_states.gather(1, idx) do?

    hidden_states shape: (batch=3, visits=4, hidden_dim=2)
    idx shape: (3, 1, 2)

How gather works:
For each patient (batch), gather takes the row (visit) along dimension 1 (the visit axis) at the index given in idx, for all hidden_dim.
Calculation for each patient
Patient 1 (batch 0):

    idx[0] = [[1, 1]]: means visit index 1, for both hidden dims.
    hidden_states[0, 1, :] = [12, 13]

Patient 2 (batch 1):

    idx[1] = [[3, 3]]: means visit index 3, for both hidden dims.
    hidden_states[1, 3, :] = [26, 27]

Patient 3 (batch 2):

    idx[2] = [[2, 2]]: means visit index 2, for both hidden dims.
    hidden_states[2, 2, :] = [34, 35]

So, after gather:
Code

selected = tensor([
    [[12, 13]],   # Patient 1
    [[26, 27]],   # Patient 2
    [[34, 35]],   # Patient 3
])  # shape: (3, 1, 2)

Step 2: Squeeze

    selected.squeeze(1) removes the singleton dimension (of size 1) at position 1.

Result:
Code

last_hidden_state = tensor([
    [12, 13],
    [26, 27],
    [34, 35],
])  # shape: (3, 2)

Summary Table
Patient	idx	Gather result	After squeeze (final)
0	[1, 1]	[[12, 13]]	[12, 13]
1	[3, 3]	[[26, 27]]	[26, 27]
2	[2, 2]	[[34, 35]]	[34, 35]





In [21]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

max_num_visits = 10
batch_size = 16
hidden_dim = 100

hidden_states = torch.randn((batch_size, max_num_visits, hidden_dim))
lengths = torch.tensor([random.randint(1, max_num_visits) for _ in range(batch_size)])
out = get_last_visit(hidden_states, lengths)

assert out.shape == (batch_size, hidden_dim)

for b in range(batch_size):
    last_h = 0
    last_h = hidden_states[b, lengths[b] - 1]
    assert torch.allclose(out[b], last_h, atol=1e-4), \
    "The last visit's hidden state of %d-th visit of the %d-th patient is wrong. "%(v,b) +\
    "Expect {} Got {} with your get_last_visit".format(last_h, out[b])



#### Build NaiveRNN [20 points]

In [22]:
import torch
import torch.nn as nn

def get_last_visit(hidden_states, length):
    idx = (length - 1).unsqueeze(1).unsqueeze(2).expand(-1, 1, hidden_states.size(2))
    last_hidden_state = hidden_states.gather(1, idx).squeeze(1)
    return last_hidden_state

class NaiveRNN(nn.Module):
    """
    Naive RNN model using GRU, Linear, and Sigmoid layers.
    """
    TOTAL_NUM_CODES = 271 
    
    def __init__(self):
        super().__init__()
        # 1. Define the RNN using nn.GRU
        self.rnn = nn.GRU(
            input_size=self.TOTAL_NUM_CODES,
            hidden_size=32,
            batch_first=True
        )
        # 2. Define the linear layer
        self.linear = nn.Linear(32, 1)
        # 3. Define the final activation layer
        self.act = nn.Sigmoid()
    
    def forward(self, x, length):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            length: the true visit length of shape (batch_size,)
        Outputs:
            probs: probabilities of shape (batch_size)
        """
        # 1. Pass the inputs through the RNN layer
        out, _ = self.rnn(x)
        # 2. Obtain the hidden state at the last visit using get_last_visit()
        last_hidden = get_last_visit(out, length)
        # 3. Pass the hidden state through the linear and activation layers
        logits = self.linear(last_hidden).squeeze(-1)  # shape: (batch_size)
        probs = self.act(logits)
        return probs

# load the model here
model = NaiveRNN()
model

NotImplementedError: 

In [ ]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

layers_to_check = [nn.GRU, nn.Linear, nn.Sigmoid]
for layer_to_check in layers_to_check:
    no_layer = True
    for child in model.children():
        for layer in child.modules():
            if(isinstance(layer, layer_to_check)):
                no_layer = False
    assert not no_layer, "{} is missing in your RNN".format(layer_to_check)

loader = DataLoader(dataset, batch_size=10, collate_fn=collate_fn)
loader_iter = iter(loader)
x, y, l = next(loader_iter)
model_output = model(x, l)
assert model_output.shape == (10,), "Your RNN's output shape is {}, expect {}".format(model_output.shape, (10,))



### Train the Network

In this step, you will train the CNN model.

In [ ]:
# Use Binary Cross Entropy as the loss function (`nn.BCELoss`)
# Use Adam as the optimizer (`torch.optim.Adam`)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

Now we can train the model.

In [ ]:
from sklearn.metrics import *

#input: Y_score,Y_pred,Y_true
#output: accuracy, auc, precision, recall, f1-score
def classification_metrics(Y_score, Y_pred, Y_true):
    acc, auc, precision, recall, f1score = accuracy_score(Y_true, Y_pred), \
                                           roc_auc_score(Y_true, Y_score), \
                                           precision_score(Y_true, Y_pred), \
                                           recall_score(Y_true, Y_pred), \
                                           f1_score(Y_true, Y_pred)
    return acc, auc, precision, recall, f1score


#input: model, loader
def evaluate(model, loader):
    model.eval()
    all_y_true = torch.LongTensor()
    all_y_pred = torch.LongTensor()
    all_y_score = torch.FloatTensor()
    for x, y, l in loader:
        # pass the input through the model
        y_hat = model(x, l)
        # convert shape from [batch size, 1] to [batch size]
        y_hat = y_hat.view(y_hat.shape[0])
        y_pred = (y_hat > 0.5).type(torch.float)
        all_y_true = torch.cat((all_y_true, y.to('cpu')), dim=0)
        all_y_pred = torch.cat((all_y_pred,  y_pred.to('cpu')), dim=0)
        all_y_score = torch.cat((all_y_score,  y_hat.to('cpu')), dim=0)
        
    acc, auc, precision, recall, f1 = classification_metrics(all_y_score.detach().numpy(), 
                                                             all_y_pred.detach().numpy(), 
                                                             all_y_true.detach().numpy())
    print(f"acc: {acc:.3f}, auc: {auc:.3f}, precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}")
    return

In [ ]:
print("model perfomance before training:")
evaluate(model, train_loader)
evaluate(model, test_loader)

In [ ]:
# number of epochs to train the model
# feel free to change this
n_epochs = 15

# prep model for training
model.train()

for epoch in range(n_epochs):
    
    train_loss = 0
    for x, y, l in train_loader:
        """ Step 1. clear gradients """
        optimizer.zero_grad()
        """  Step 2. perform forward pass using `model`, save the output to y_hat """
        y_hat = model(x, l)
        """ Step 3. calculate the loss using `criterion`, save the output to loss. """
        # convert shape from [batch size, 1] to [batch size]
        y_hat = y_hat.view(y_hat.shape[0])
        loss = criterion(y_hat, y)
        """ Step 4. backward pass """
        loss.backward()
        """ Step 5. optimization """
        optimizer.step()
        """ Step 6. record loss """
        train_loss += loss.item()
        
    train_loss = train_loss / len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
    evaluate(model, train_loader)
    evaluate(model, test_loader)

The result is bad due to very limited data. The model overfits the training data very fast.

You are encouraged to try this on the whole MIMIC-III dataset. The result will be much more promising!